# Titanic Survival Classification - Learning Concepts  - Decision Tree (Part 7)

After trying various combinations of input variables and neural network architectures and hyperparameters, every model seems to converge in approximately the same place - 
* 80-85% Training Accuracy
* 80-85% Cross Validation Accuracy
* 75-78% Test (Submission) Accuracy

So clearly there is something more productive to increase accuracy rather than endlessly iterating over different neural networks.  

In light of this I went back and did some research into the models that had vastly different performance - Gradient Boosting/Random Forest.  These models heavily overfit the data but they did achieve as yet unseen levels of training accuracy so I wanted to understand why the models performed so differently and the results of that research are as follows.

## Decision Tree Classifiers

*Note all of below is based on online research and therefore may be incorrect/incomplete*

So a decision tree is at its simplest a series of if-then-else conditional logical statements which segment the data.  For example in this set the logical starting place would be if male/female.  Then splitting the male/female by Pclass each and so on and so forth.  

The surived/not survived ratio at any point in the split is usually measured by a chance of misclassifying at that level and is effectively a sum of probabilities (ie probability of survived and misclassifying + probability of died and misclassifying), called the Gini Impurity.

Random forests are simply ensembles of decision trees segmented randomly by variable selection and in some instances by the data rows trained on.

Gradient Boosting I am not 100% sure on how it works but as far as I can tell its training a simple decision tree with a very limited depth and taking the observations that this tree is bad at classifying and boosting their priority and training another weak decision tree and iterating upon that process.  The final output being an ensemble model which can be averaged to give a prediction.

The reason my experimental models overfit in earlier parts is because they had learned to segment by random noise in the data and hence were excellent at fitting the training data but did not generalize at all.  Unfortunately from what I can tell, due to this linear-logical structure there is no simple way to regularize other than limiting the tree depth and fiddling with hyperparameters and implementing early stopping witha  cross validation prediction cost.  There is not a equivalent of the $\lambda$ which you can turn up and down as in L2/L1 regularization in regression based models.


## Dual Path Network

In light of this the next avenue to approach is a dual path network to get a neural network specialized in dealing with the examples that the first network struggled with.  Its not quite the same principle as gradient boosting but I do not yet know enough about gradient boosting to be able to apply the principle to neural networks (if it is at all possible).


The idea is train a network as we have done previously and then get the misclassified examples, and train two more networks. 
* A discriminator network to determine which branch the training example should follow, so using the classification output of network 1, creating a new $\hat{y}$ mask as accurate or not.
* A secondary classification network trained on the examples the first network misclassified.

The potential problem I see is not having enough training examples for the secondary network to converge, however its an interesting enough idea it is worth testing.


In [1]:
#First importing some relevant packages
import numpy as np
import pandas as pd

#Import Tensorflow
import tensorflow as tf

#Import Keras
from keras import layers
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Sequential, Model
from keras import regularizers
from keras.optimizers import Adam, SGD

#Import mathematical functions
from random import *
import math
import matplotlib
import matplotlib.pyplot as plt

#Get regular expression package
import re

#Import  Scikit learn framework
import sklearn as sk
from sklearn import svm
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)

F:\anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#Import the functions built in previous parts
from Titanic_Import import *

full_set = pd.read_csv('D:/Datasets/Titanic/train.csv')
sub_set = pd.read_csv('D:/Datasets/Titanic/test.csv')

In [4]:
append_set = full_set
append_set = append_set.append([sub_set], ignore_index =True )
clean_set = Cleanse_Data_v3(append_set)
X_Train, Y_Train, X_CV, Y_CV, X_Test = dataset_splitter(clean_set, cv_size = 200)

So the first step is to build the first model.  So lets build a simple neural network as usual.

In [4]:
layers = [21, 17, 14, 8, 5, 5]

In [5]:
first_model = NN_model((X_Train.shape[1], ), layers, regularizers.l2(0.013), None)
first_model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])
first_model.fit(x = X_Train, y = Y_Train, epochs = 256, verbose = 1)

Epoch 1/256
691/691 [==============================] - 35s 51ms/step - loss: 2.5303 - acc: 0.6194
Epoch 2/256
691/691 [==============================] - 0s 289us/step - loss: 1.3484 - acc: 0.6194
Epoch 3/256
691/691 [==============================] - 0s 302us/step - loss: 0.9471 - acc: 0.6194
Epoch 4/256
691/691 [==============================] - 0s 308us/step - loss: 0.7919 - acc: 0.6194
Epoch 5/256
691/691 [==============================] - 0s 297us/step - loss: 0.7325 - acc: 0.6194
Epoch 6/256
691/691 [==============================] - 0s 301us/step - loss: 0.7105 - acc: 0.6194
Epoch 7/256
691/691 [==============================] - 0s 294us/step - loss: 0.6973 - acc: 0.6194
Epoch 8/256
691/691 [==============================] - 0s 295us/step - loss: 0.6898 - acc: 0.6194
Epoch 9/256
691/691 [==============================] - 0s 298us/step - loss: 0.6855 - acc: 0.6194
Epoch 10/256
691/691 [==============================] - 0s 304us/step - loss: 0.6813 - acc: 0.6194
Epoch 11/256
691/69

691/691 [==============================] - 0s 294us/step - loss: 0.3836 - acc: 0.8871
Epoch 166/256
691/691 [==============================] - 0s 295us/step - loss: 0.3799 - acc: 0.8871
Epoch 167/256
691/691 [==============================] - 0s 295us/step - loss: 0.3911 - acc: 0.8741
Epoch 168/256
691/691 [==============================] - 0s 300us/step - loss: 0.3808 - acc: 0.8828
Epoch 169/256
691/691 [==============================] - 0s 298us/step - loss: 0.3906 - acc: 0.8799
Epoch 170/256
691/691 [==============================] - 0s 288us/step - loss: 0.3843 - acc: 0.8871
Epoch 171/256
691/691 [==============================] - 0s 289us/step - loss: 0.3864 - acc: 0.8842
Epoch 172/256
691/691 [==============================] - 0s 294us/step - loss: 0.3927 - acc: 0.8698
Epoch 173/256
691/691 [==============================] - 0s 291us/step - loss: 0.3816 - acc: 0.8784
Epoch 174/256
691/691 [==============================] - 0s 304us/step - loss: 0.3897 - acc: 0.8726
Epoch 175/256


691/691 [==============================] - 0s 289us/step - loss: 0.3609 - acc: 0.8857
Epoch 248/256
691/691 [==============================] - 0s 294us/step - loss: 0.3720 - acc: 0.8755
Epoch 249/256
691/691 [==============================] - 0s 295us/step - loss: 0.3567 - acc: 0.8828
Epoch 250/256
691/691 [==============================] - 0s 305us/step - loss: 0.3534 - acc: 0.8886
Epoch 251/256
691/691 [==============================] - 0s 282us/step - loss: 0.3577 - acc: 0.8813
Epoch 252/256
691/691 [==============================] - 0s 291us/step - loss: 0.3638 - acc: 0.8799
Epoch 253/256
691/691 [==============================] - 0s 285us/step - loss: 0.3604 - acc: 0.8813
Epoch 254/256
691/691 [==============================] - 0s 279us/step - loss: 0.3556 - acc: 0.8842
Epoch 255/256
691/691 [==============================] - 0s 291us/step - loss: 0.3550 - acc: 0.8871
Epoch 256/256
691/691 [==============================] - 0s 292us/step - loss: 0.3559 - acc: 0.8813


In [6]:
train_pred = first_model.predict(x = X_Train)
cv_pred = first_model.predict(x = X_CV)

In [7]:
train_hat = normalize_predictions(train_pred)
cv_hat = normalize_predictions(cv_pred)

In [8]:
acc1, score1, conf1 = Calc_Accuracy(Y_Train, train_hat)

print("Accuracy = ", acc1)
print("F1 Score = ", score1)
print("")
print("Confusion Matrix")
conf1[["Labels", "Actual True", "Actual False"]]

Accuracy =  89.00144717800289
F1 Score =  0.8492063492063492

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,214.0,27.0
1,Pred False,49.0,401.0


In [9]:
acc2, score2, conf2 = Calc_Accuracy(Y_CV, cv_hat)

print("Accuracy = ", acc2)
print("F1 Score = ", score2)
print("")
print("Confusion Matrix")
conf2[["Labels", "Actual True", "Actual False"]]

Accuracy =  78.0
F1 Score =  0.7215189873417721

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,57.0,22.0
1,Pred False,22.0,99.0


Performance is about as expected.  So next step is to build a classifier to identify the misclassified examples.

So to do this we simply have to create our new Y which will be 1 in the case of our fist classifier being correct and 0 otherwise.

In [10]:
Y_determ_pre = np.absolute(np.ones(Y_Train.shape) - Y_Train - train_hat)
Y_det_CV = np.absolute(np.ones(Y_CV.shape) - Y_CV - cv_hat)

However due to the extreme data (after all our model was 80% accurate) this means we need to address the data mismatch as optimizing across accuracy leads to the model attributing every output a prediction of 1.  Hence we need to augment the training data a bit.

In [12]:
Y_determ_pre.shape

(691,)

In [78]:
#Get all indices where Y is incorrect
train_ind = np.argwhere(Y_determ_pre == 0)
train_ind = np.squeeze(train_ind)

#Pulll all rows from X where Y is incorrect
determ_ones_train = X_Train[train_ind, :]


#Get indices where Y is correct
not_in_train = [x for x in range(len(X_Train)) if x not in train_ind]

#Pull all data where Y is correct
all_zeros_train = X_Train[not_in_indices, :]


#Sample top X where Y is correct to get 50/50 training split
determ_zeros_train = all_zeros_train[:76, :]

#Append both datasets together to get  final training set
determ_train = np.vstack((determ_ones_train, determ_zeros_train))
#Create Y vector for answers
determ_Y = np.hstack((np.ones(76,), np.zeros(76,)))

Now to train a new model to determine which path the prediction should use.

In [96]:
layers = [23, 17, 10, 7, 5, 3]
determ_model = NN_model((determ_train.shape[1], ), layers, None, None)
determ_model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])
determ_model.fit(x = determ_train, y = determ_Y, epochs = 1024, verbose = 1)

Epoch 1/1024
152/152 [==============================] - 1s 6ms/step - loss: 0.7278 - acc: 0.5000
Epoch 2/1024
152/152 [==============================] - 0s 276us/step - loss: 0.7044 - acc: 0.5000
Epoch 3/1024
152/152 [==============================] - 0s 276us/step - loss: 0.6965 - acc: 0.5066
Epoch 4/1024
152/152 [==============================] - 0s 283us/step - loss: 0.6939 - acc: 0.5197
Epoch 5/1024
152/152 [==============================] - 0s 270us/step - loss: 0.6929 - acc: 0.5263
Epoch 6/1024
152/152 [==============================] - 0s 289us/step - loss: 0.6907 - acc: 0.5395
Epoch 7/1024
152/152 [==============================] - 0s 309us/step - loss: 0.6885 - acc: 0.5395
Epoch 8/1024
152/152 [==============================] - 0s 296us/step - loss: 0.6862 - acc: 0.5724
Epoch 9/1024
152/152 [==============================] - 0s 289us/step - loss: 0.6843 - acc: 0.5789
Epoch 10/1024
152/152 [==============================] - 0s 289us/step - loss: 0.6832 - acc: 0.5724
Epoch 11/10

152/152 [==============================] - 0s 276us/step - loss: 0.5241 - acc: 0.6579
Epoch 164/1024
152/152 [==============================] - 0s 283us/step - loss: 0.5039 - acc: 0.6908
Epoch 165/1024
152/152 [==============================] - 0s 283us/step - loss: 0.5118 - acc: 0.6776
Epoch 166/1024
152/152 [==============================] - 0s 283us/step - loss: 0.5114 - acc: 0.7039
Epoch 167/1024
152/152 [==============================] - 0s 276us/step - loss: 0.5066 - acc: 0.6908
Epoch 168/1024
152/152 [==============================] - 0s 309us/step - loss: 0.4991 - acc: 0.7039
Epoch 169/1024
152/152 [==============================] - 0s 296us/step - loss: 0.5104 - acc: 0.6711
Epoch 170/1024
152/152 [==============================] - 0s 283us/step - loss: 0.4979 - acc: 0.7237
Epoch 171/1024
152/152 [==============================] - 0s 289us/step - loss: 0.4984 - acc: 0.7237
Epoch 172/1024
152/152 [==============================] - 0s 303us/step - loss: 0.4940 - acc: 0.7039
Epoch

152/152 [==============================] - 0s 276us/step - loss: 0.4420 - acc: 0.7632
Epoch 326/1024
152/152 [==============================] - 0s 257us/step - loss: 0.4568 - acc: 0.7368
Epoch 327/1024
152/152 [==============================] - 0s 276us/step - loss: 0.4546 - acc: 0.7434
Epoch 328/1024
152/152 [==============================] - 0s 263us/step - loss: 0.4459 - acc: 0.7368
Epoch 329/1024
152/152 [==============================] - 0s 257us/step - loss: 0.4416 - acc: 0.7763
Epoch 330/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4428 - acc: 0.7632
Epoch 331/1024
152/152 [==============================] - 0s 257us/step - loss: 0.4455 - acc: 0.7500
Epoch 332/1024
152/152 [==============================] - 0s 283us/step - loss: 0.4442 - acc: 0.7632
Epoch 333/1024
152/152 [==============================] - 0s 263us/step - loss: 0.4470 - acc: 0.7171
Epoch 334/1024
152/152 [==============================] - 0s 263us/step - loss: 0.4546 - acc: 0.7566
Epoch

152/152 [==============================] - 0s 270us/step - loss: 0.4316 - acc: 0.7368
Epoch 488/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4214 - acc: 0.7500
Epoch 489/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4209 - acc: 0.7895
Epoch 490/1024
152/152 [==============================] - 0s 263us/step - loss: 0.4172 - acc: 0.7763
Epoch 491/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4238 - acc: 0.7632
Epoch 492/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4257 - acc: 0.7434
Epoch 493/1024
152/152 [==============================] - 0s 270us/step - loss: 0.4143 - acc: 0.7632
Epoch 494/1024
152/152 [==============================] - 0s 263us/step - loss: 0.4176 - acc: 0.7566
Epoch 495/1024
152/152 [==============================] - 0s 303us/step - loss: 0.4129 - acc: 0.7697
Epoch 496/1024
152/152 [==============================] - 0s 276us/step - loss: 0.4172 - acc: 0.7434
Epoch

152/152 [==============================] - 0s 257us/step - loss: 0.3880 - acc: 0.7697
Epoch 650/1024
152/152 [==============================] - 0s 276us/step - loss: 0.4131 - acc: 0.7697
Epoch 651/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3901 - acc: 0.7763
Epoch 652/1024
152/152 [==============================] - 0s 276us/step - loss: 0.3942 - acc: 0.7763
Epoch 653/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3983 - acc: 0.7697
Epoch 654/1024
152/152 [==============================] - 0s 276us/step - loss: 0.3940 - acc: 0.7566
Epoch 655/1024
152/152 [==============================] - 0s 263us/step - loss: 0.3875 - acc: 0.7895
Epoch 656/1024
152/152 [==============================] - 0s 257us/step - loss: 0.3890 - acc: 0.8026
Epoch 657/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3978 - acc: 0.7632
Epoch 658/1024
152/152 [==============================] - 0s 263us/step - loss: 0.3876 - acc: 0.7961
Epoch

152/152 [==============================] - 0s 283us/step - loss: 0.3713 - acc: 0.7829
Epoch 812/1024
152/152 [==============================] - 0s 289us/step - loss: 0.3731 - acc: 0.7961
Epoch 813/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3854 - acc: 0.7895
Epoch 814/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3691 - acc: 0.7895
Epoch 815/1024
152/152 [==============================] - 0s 270us/step - loss: 0.3704 - acc: 0.8092
Epoch 816/1024
152/152 [==============================] - 0s 283us/step - loss: 0.3871 - acc: 0.7697
Epoch 817/1024
152/152 [==============================] - 0s 322us/step - loss: 0.3759 - acc: 0.8026
Epoch 818/1024
152/152 [==============================] - 0s 283us/step - loss: 0.3662 - acc: 0.8026
Epoch 819/1024
152/152 [==============================] - 0s 316us/step - loss: 0.3715 - acc: 0.7829
Epoch 820/1024
152/152 [==============================] - 0s 322us/step - loss: 0.3784 - acc: 0.7763
Epoch

152/152 [==============================] - 0s 289us/step - loss: 0.3538 - acc: 0.8158
Epoch 974/1024
152/152 [==============================] - 0s 283us/step - loss: 0.3667 - acc: 0.8026
Epoch 975/1024
152/152 [==============================] - 0s 296us/step - loss: 0.3689 - acc: 0.7697
Epoch 976/1024
152/152 [==============================] - 0s 289us/step - loss: 0.3526 - acc: 0.8026
Epoch 977/1024
152/152 [==============================] - 0s 289us/step - loss: 0.3538 - acc: 0.8158
Epoch 978/1024
152/152 [==============================] - 0s 289us/step - loss: 0.3564 - acc: 0.8092
Epoch 979/1024
152/152 [==============================] - 0s 303us/step - loss: 0.3514 - acc: 0.8026
Epoch 980/1024
152/152 [==============================] - 0s 296us/step - loss: 0.3566 - acc: 0.7895
Epoch 981/1024
152/152 [==============================] - 0s 303us/step - loss: 0.3504 - acc: 0.8158
Epoch 982/1024
152/152 [==============================] - 0s 289us/step - loss: 0.3577 - acc: 0.7895
Epoch

In [97]:
train_pred_det = determ_model.predict(determ_train)
cv_pred_det = determ_model.predict(X_CV)
train_hat_det = normalize_predictions(train_pred_det)
cv_hat_det = normalize_predictions(cv_pred_det)

In [98]:
acc1, score1, conf1 = Calc_Accuracy(determ_Y, train_hat_det)

print("Accuracy = ", acc1)
print("F1 Score = ", score1)
print("")
print("Confusion Matrix")
conf1[["Labels", "Actual True", "Actual False"]]

Accuracy =  76.31578947368422
F1 Score =  0.8043478260869565

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,74.0,34.0
1,Pred False,2.0,42.0


In [99]:
acc2, score2, conf2 = Calc_Accuracy(Y_det_CV, cv_hat_det)

print("Accuracy = ", acc2)
print("F1 Score = ", score2)
print("")
print("Confusion Matrix")
conf2[["Labels", "Actual True", "Actual False"]]

Accuracy =  53.5
F1 Score =  0.6568265682656826

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,89.0,26.0
1,Pred False,67.0,18.0


So sadly our discriminator function is not very good at discriminating, even when the data is balanced, I suspect there is simply not enough training data for a good model to be built here.

When the function is trained on the full training sample the data is so mismatched, perhaps changing the optimization to optimize along Precision or Recall instead of Accuracy may improve performance on mismatched data, however this intuitively feels like a bit of a dead end to continue persuing.

However from this I am now confident I am able to implement a dual path network in the future so maybe this skill will come in useful in future projects.

# Decision Tree Classifier from scratch

So as I have very little experience with Decision Tree based models the best logical place to start is to create a simple decision tree model from scratch.

Note all the below is loosely based on the following sources - 

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

https://gormanalysis.com/magic-behind-constructing-a-decision-tree/

So how it works from my basic understanding - 
1. Pick variable to split
2. Split data along variable  (eg if binary one branch for 1, another for 0)
3. Calculate Gini index (measure of impurity - formula to follow)
4. Generate the best split as one with lowest Gini
5. Repeat 1-4 to generate full tree

## Gini Impurity 

So the calculation for the gini impurity is as follows - 

$i(t) = 1 - \Sigma^k_{j=1}{p^2 (j|t)}$

However lets break down what this actually means.

So at every potential output k (so for binary column k = 2) each node will have a probability p of selecting an observation j being of class t.

For a more concrete example, lets say we arbitrarily split our data by a random binary variable (i.e. k = 2, j = 1, 2) if we have a gini of 0.5 we will have an exactly 50/50 chance of selecting a survivor, ie each branch has an exactly even amount of survived vs not survived.|

So lets take a worked example - the first 10 rows of our training set.

In [822]:
#Picked 6th variable as it was an imperfect classifier and was binary for these observations
train_gini = X_Train[0:10, 10]
print(train_gini)

[1. 0. 0. 1. 1. 1. 0. 0. 0. 0.]


In [627]:
y_gini = Y_Train[0:10]
print(y_gini)

[0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]


So here we have - 

X | Y | Number
--- | --- | ---
0 | 0 | 1
0 | 1 | 5
1 | 0 | 4
1 | 1 | 0

Hence for X = 0 we have $\frac{1}{6} * \frac{5}{6} + \frac{5}{6} * \frac{1}{6}  = \frac{5}{36} + \frac{5}{36} = \frac{5}{18} $

Similarly for X = 1 we have $\frac{4}{4} * \frac{0}{4} + \frac{0}{4} * \frac{4}{4} = 0 $


So our final gini score we weight by the fraction in each leaf and sum  as such - 

$\frac{5}{18} * \frac{6}{10} + 0 * \frac{4}{10} = \frac{1}{6} = 0.17$

Which is a low score meaning this split is pretty good at seperating our data.

First up we need a function to split our data into n sections along any column

In [768]:
def Split_data(X_in, Y_in, col=0, n_outputs=2):
    outputs_X = []
    outputs_Y = []
    threshholds = np.zeros(n_outputs)
    

    if len(X_in.shape) == 1 :
        X_max = max(X_in)
        X_min = min(X_in)
        for i in range(n_outputs):
            #For simplicity leaving it as evenly segmenting over selected variable range
            #This should suffice due to normalizing my inputs
            threshholds[i] = ((X_max - X_min) / n_outputs)*(i + 1) + X_min
            if i == (n_outputs - 1):
                threshholds[i] = X_max
            
        for j in range(n_outputs):
            if j == 0:
                output_mat_X = X_in[np.where(X_in <= threshholds[j])]
                output_mat_Y = Y_in[np.where(X_in <= threshholds[j])]                       
            else:  
                output_mat_X = X_in[np.where(np.logical_and(threshholds[j - 1]<  X_in, X_in <= threshholds[j] ))]
                #output_mat_X = X_in[np.where(threshholds[j - 1]<=  X_in && X_in < threshholds[j])]
                output_mat_Y = Y_in[np.where(np.logical_and(threshholds[j - 1]<  X_in, X_in <= threshholds[j] ))]                
                                    
            outputs_X.append(output_mat_X)
            outputs_Y.append(output_mat_Y)
            
    else:
        X_max = max(X_in[:, col])
        X_min = min(X_in[:, col])
        for i in range(n_outputs):
            threshholds[i] = ((X_max - X_min) / n_outputs)*(i + 1) + X_min
            
        if i == (n_outputs - 1):
                threshholds[i] = X_max
        
        for j in range(n_outputs):
            if j == 0:
                output_mat_X = X_in[np.where(X_in[:, col] <= threshholds[j])]
                output_mat_Y = Y_in[np.where(X_in[:, col] <= threshholds[j])]                       
            else:  
                output_mat_X = X_in[np.where(np.logical_and(threshholds[j - 1]<  X_in[:, col] , X_in[:, col]  <= threshholds[j] ))]
                output_mat_Y = Y_in[np.where(np.logical_and(threshholds[j - 1]<  X_in[:, col] , X_in[:, col] <= threshholds[j] ))]                
                                    
            outputs_X.append(output_mat_X)
            outputs_Y.append(output_mat_Y)
    
    
    return outputs_X, outputs_Y, threshholds


In [769]:
testX, testY, threshholds = Split_data(train_gini, y_gini, 10,2)

print(testX[0])
print(testY[0])

[0. 0. 0. 0. 0. 0.]
[1. 1. 0. 1. 1. 1.]


So looking at all our values where the X was 0, the Y pattern does match up meaning our splitter works correctly.

Hence next up is to use this to calculate our Gini Impurity as above.


In [488]:
def calc_gini(Y_inputs):
    num_datasets = len(Y_inputs)
    denom = np.zeros(len(Y_inputs))
    num_ones = np.zeros(len(Y_inputs))
    num_zeros = np.zeros(len(Y_inputs))
    predictions = np.zeros(len(Y_inputs))
    
    gini = 0.0
    
    num_obs = 0
    
    for i in range(num_datasets):
        Y_data = Y_inputs[i]
        denom[i] = len(Y_data)
        num_ones[i] = sum(Y_data)
        num_zeros[i] = denom[i] - num_ones[i]
        num_obs += denom[i]
          
    for j in range(num_datasets):    
        leaf_prop = denom[j] / num_obs
        leaf_gini = ((num_ones[j]/ denom[j]) * (num_ones[j]/ denom[j])) + ((num_zeros[j] / denom[j]) *(num_zeros[j] / denom[j]))
        predictions[j] = (num_ones[j]/ denom[j]) 

        
        gini += leaf_gini*leaf_prop
        
    gini = 1 - gini
    return gini, predictions

In [489]:
test_gin, preds = calc_gini(testY)
print(test_gin)

0.16666666666666652


This matches what we calculated indicating our gini calculation function works correctly!

Next we need a function to find the best variable to split on by randomly testing and finding the one that gives the lowest gini.

In [783]:
def find_split(X_input, Y_input, max_iters=2, max_children = 2):
    X_splits = []
    Y_splits = []
    results_cache = {}
    gini = 0.5
    error = 0.5
    no_split = 0
    
    sum_Y = sum(Y_input)
    
    if sum_Y == 0 :
        results_cache['gini'] =  0
        results_cache['predictions'] =  0
        results_cache['split_col'] = 9
        results_cache['num_childs'] = 0
        results_cache['thresholds'] = 0
        return X_splits, Y_splits, results_cache
    
    if int(sum_Y) == len(Y_input) :
        results_cache['gini'] =  0
        results_cache['predictions'] =  1
        results_cache['split_col'] = 9
        results_cache['num_childs'] = 0
        results_cache['thresholds'] = 0
        return X_splits, Y_splits, results_cache
    
    num_vars = X_input.shape[1] - 1
    
    for i in range(max_iters):
        test_col = randint(0, num_vars)
        if max_children == 2:
            num_childs = 2
        else :
            num_childs = randint(2, max_children)
        
        #Generate split
        testX, testY, thresh = Split_data(X_input, Y_input, test_col, num_childs)
        
        #Generate another split if results in null branch
        error = 0.5
        while error > 0 :
            n = 0
            for j in range(len(testY)) : 
                if len(testY[j]) == 0:
                    error += 1
                    
            if error == 0.5:
                error = 0
                break
            else :
                test_col = randint(0, num_vars)
                testX, testY, thresh = Split_data(X_input, Y_input, test_col, 2)
                num_childs = 2
                error = 0.5
                n += 1
            
            #Add non-convergence criteria
            if n >= 15 :
                error = 0
                print('Error did not split')
                no_split = 1
                break
        
        #Calculate Gini
        if no_split == 0 :
          test_gin, preds = calc_gini(testY)
        else :
            #Account for non-convergence
            test_gin = 0.5
            preds = 0.5
            X_splits.append(X_input)
            Y_splits.append(Y_input)
        
        if i == 0 :
            X_splits = testX
            Y_splits = testY
            results_cache['gini'] =  test_gin
            results_cache['predictions'] =  preds
            results_cache['split_col'] = test_col
            results_cache['num_childs'] = num_childs
            results_cache['thresholds'] = thresh
        elif test_gin < results_cache['gini'] :
            X_splits = testX
            Y_splits = testY
            results_cache['gini'] =  test_gin
            results_cache['predictions'] =  preds
            results_cache['split_col'] = test_col
            results_cache['num_childs'] = num_childs
            results_cache['thresholds'] = thresh
    
    return X_splits, Y_splits, results_cache

In [785]:
X_gini_test = X_Train[0:10, :]

X_split_test, Y_split_test, cache = find_split(X_gini_test,y_gini, 10, 4)
print(cache)

{'gini': 0.16666666666666652, 'predictions': array([0.83333333, 0.        ]), 'split_col': 10, 'num_childs': 2, 'thresholds': array([0.5, 1. ])}


So we now have a function to get a single best-split on one part of the data.  So to turn this into a full tree we need to iterate the above function from the start and then again on each branch until a specified depth or the overall gini is 0.

To do this a recursive tree generator will be used and parameters passed between layers to keep track of the history, which can then be used to retro-actively build the tree when predicting.

In [971]:
def Generate_Tree(X_input, Y_input, model, cur_depth = 0 , child_num = -1, max_depth = 10, max_childs = 3, max_iters = 10):
    mod = {}
    
    if cur_depth >= max_depth :
        return model, cur_depth
    
    X_split, Y_split, cache = find_split(X_input,Y_input, max_iters, max_childs)
    
    col = cache['split_col']
    childs = cache['num_childs']
    preds = cache['predictions']
    thresh = cache['thresholds']
    
    if childs == 0 :
        #mod['prediction'] = preds
        #model.append(mod)
        #mod = {}
        return model, cur_depth
    
    for n in range(childs) :
        X_new = X_split[n]
        Y_new =Y_split[n]
        index = len(model)
        mod['index'] = index
        mod['Layer'] = cur_depth
        mod['split_col'] = col
        mod['prev_parent'] = child_num
        #mod['hist'] = hist
        
        
        if n == 0:
            mod['threshmin'] = -99.99
            mod['threshmax'] = thresh[n]
        else :
            mod['threshmin'] = thresh[n - 1]
            mod['threshmax'] = thresh[n]
            
        mod['prediction'] = preds[n]
        
        model.append(mod)
        mod = {}
        depth = cur_depth + 1
        Generate_Tree(X_new,Y_new, model, depth,index, max_depth, max_childs, max_iters)
        
    
    return model, cur_depth

In [972]:
#model_test = Generate_Tree(X_gini_test, y_gini, max_depth = 2, max_childs = 2, max_iters = 10)
model_test , d = Generate_Tree(X_gini_test,y_gini, [], max_depth = 4, max_childs = 2, max_iters = 10)

In [973]:
model_test

[{'Layer': 0,
  'index': 0,
  'prediction': 0.2857142857142857,
  'prev_parent': -1,
  'split_col': 17,
  'threshmax': 0.5,
  'threshmin': -99.99},
 {'Layer': 1,
  'index': 1,
  'prediction': 1.0,
  'prev_parent': 0,
  'split_col': 1,
  'threshmax': 0.5,
  'threshmin': -99.99},
 {'Layer': 1,
  'index': 2,
  'prediction': 0.0,
  'prev_parent': 0,
  'split_col': 1,
  'threshmax': 1.0,
  'threshmin': 0.5},
 {'Layer': 0,
  'index': 3,
  'prediction': 1.0,
  'prev_parent': -1,
  'split_col': 17,
  'threshmax': 1.0,
  'threshmin': 0.5}]

So from these nodes we have all the information encoded which we can use to generate predictions.

In [1009]:
def Gen_Predictions(X_input, model):
    num_obs = X_input.shape[0]
    num_nodes = len(model)

    #Initialize vectors for data extract
    indices = np.zeros(num_nodes)
    prev_index = np.zeros(num_nodes)
    prediction = np.zeros(num_nodes)
    col = np.zeros(num_nodes)
    threshmax = np.zeros(num_nodes)
    threshmin = np.zeros(num_nodes)
    Layer = np.zeros(num_nodes)
    
    #Initialize Y_hat
    Y_hat = np.zeros(num_obs)
    
    #testing
    for y in range(len(Y_hat)) :
        Y_hat[y] = 9.9

    #Extract model data into useable form
    for n in range(num_nodes) :
        mod = model[n]
        indices[n] = mod['index']
        prev_index[n] = mod['prev_parent']
        prediction[n] = mod['prediction']
        col[n] = mod['split_col']
        threshmax[n] = mod['threshmax']
        threshmin[n] = mod['threshmin']
        Layer[n] = mod['Layer'] 

    #Get meta vars for parsing
    num_layers = int(max(Layer) + 1)
    
    #Iterate predictions
    for m in range(num_obs):
        X_row = X_input[m, :]
        #Initialize first layer as layer -1
        prev_ind = -1
        
        for l in range(num_layers) :
            #Get indices of layer by comparing to previous
            prev_i = [i for i,x in enumerate(prev_index) if x == prev_ind]
            
            for idx in prev_i :
                column = int(col[idx])
                max_t = threshmax[idx]
                min_t = threshmin[idx]
                
                if X_row[column] > min_t and X_row[column] <= max_t:
                    Y_hat[m] = prediction[idx]
                    prev_ind = indices[idx]
         
    
    return Y_hat

In [1013]:
y_test = Gen_Predictions(X_gini_test, model_test)

In [1014]:
y_test

array([0., 1., 1., 0., 0., 0., 0., 1., 1., 1.])

In [1015]:
y_gini

array([0., 1., 1., 0., 0., 0., 0., 1., 1., 1.])

So from the above test it seems like our model works nicely on a small scale.  Lets extend this model and see how it works on a full dataset.

In [1018]:
model_test2, d = Generate_Tree(X_Train,Y_Train, [], max_depth = 10, max_childs = 4, max_iters = 10)

In [1020]:
y_test2 = Gen_Predictions(X_gini_test, model_test2)

In [1021]:
y_test2

array([0.11, 1.  , 1.  , 0.11, 0.11, 0.  , 0.  , 1.  , 1.  , 1.  ])

Looking at this we have fairly accurately modelled our test vector.  Plus all of the predictions that our model is 100% confident on are in fact correct.  

This is a positive sign as it should be as we are testing how well it has fit our training data, for cross validation it would not be expected that this would be the case.

So lets see how it does overall and in cross validation.

In [1026]:
train_tree_y = normalize_predictions(Gen_Predictions(X_Train, model_test2))
cv_tree_y = normalize_predictions(Gen_Predictions(X_CV, model_test2))

In [1027]:
acc4, score4, conf4 = Calc_Accuracy(Y_Train, train_tree_y)

print("Accuracy = ", acc4)
print("F1 Score = ", score4)
print("")
print("Confusion Matrix")
conf4[["Labels", "Actual True", "Actual False"]]

Accuracy =  93.05354558610709
F1 Score =  0.9040000000000001

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,226.0,8.0
1,Pred False,40.0,417.0


In [1028]:
acc5, score5, conf5 = Calc_Accuracy(Y_CV, cv_tree_y)

print("Accuracy = ", acc5)
print("F1 Score = ", score5)
print("")
print("Confusion Matrix")
conf5[["Labels", "Actual True", "Actual False"]]

Accuracy =  78.5
F1 Score =  0.6950354609929077

Confusion Matrix


,Labels,Actual True,Actual False
0,Pred True,49.0,16.0
1,Pred False,27.0,108.0


This is not bad at all, it has slightly overfit but by comparison to the logistic regression function built from scratch this model has performed admirably (the logistic regression had ~80% train + CV accuracy).

I am certain this is not the most efficient implementation, and I could put into an object oriented framework to make it easier to use.  I'm certain a more memory efficient model representation and a vectorized implementation of the predict function would be much better, plus I could extent the model by expanding it to regression as well as binary classification or perhaps a random forest or other such ensemble. 

However, the purpose of doing this was to gain a better understanding of how decision tree based models work, which is an objective I have accomplished in this notebook. 

To follow is to build a Gradient Boosting Machine from scratch and maybe see if an autoencoder can perform better feature extraction than I have.

Part 8 to follow.